Modified code from [here](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/03-data-warehouse/extras/web_to_gcs.py)

In [19]:
import io
import os
import requests
import pandas as pd
from google.cloud import storage

In [36]:
"""
Pre-reqs: 
1. `pip install pandas pyarrow google-cloud-storage`
2. Set GOOGLE_APPLICATION_CREDENTIALS to your project/service-account key
3. Set GCP_GCS_BUCKET as your bucket or change default value of BUCKET
"""

# services = ['fhv','green','yellow']
init_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/'

creds = '/Users/elijahsutton/Documents/projects/de-zoomcamp-2024/module_1/1_terraform_gcp/keys/creds.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = creds

BUCKET = os.environ.get('GCP_GCS_BUCKET', 'de_zoomcamp_mage_18469')

In [37]:
def upload_to_gcs(bucket, object_name, local_file):
    """
    Ref: https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
    """
    # # WORKAROUND to prevent timeout for files > 6 MB on 800 kbps upload speed.
    # # (Ref: https://github.com/googleapis/python-storage/issues/74)
    storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB
    storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB

    client = storage.Client()
    bucket = client.bucket(bucket)
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)

In [38]:
def web_to_gcs(year, service):
    for i in range(12):
        
        # sets the month part of the file_name string
        month = '0'+str(i+1)
        month = month[-2:]

        # csv file_name
        file_name = f"{service}_tripdata_{year}-{month}.csv.gz"

        # download it using requests via a pandas df
        request_url = f"{init_url}{service}/{file_name}"
        r = requests.get(request_url)
        open(file_name, 'wb').write(r.content)
        # os.system(f'curl -L {request_url} -o {file_name}')
        print(f"Local: {file_name}")

        # read it back into a parquet file
        df = pd.read_csv(file_name, compression='gzip')
        file_name = file_name.replace('.csv.gz', '.parquet')
        df.to_parquet(file_name, engine='pyarrow')
        print(f"Parquet: {file_name}")

        # upload it to gcs 
        upload_to_gcs(BUCKET, f"{service}/{file_name}", file_name)
        print(f"GCS: {service}/{file_name}")

        # Remove compressed CSV and parquet files locally
        file_name_no_ext = file_name.split('.')[0]
        print('Removing locally:')
        os.system(f'rm -v {file_name_no_ext}.*')

In [39]:
web_to_gcs('2019', 'fhv')

Local: fhv_tripdata_2019-01.csv.gz
Parquet: fhv_tripdata_2019-01.parquet
GCS: fhv/fhv_tripdata_2019-01.parquet
Removing locally:
fhv_tripdata_2019-01.csv.gz
fhv_tripdata_2019-01.parquet
Local: fhv_tripdata_2019-02.csv.gz
Parquet: fhv_tripdata_2019-02.parquet
GCS: fhv/fhv_tripdata_2019-02.parquet
Removing locally:
fhv_tripdata_2019-02.csv.gz
fhv_tripdata_2019-02.parquet
Local: fhv_tripdata_2019-03.csv.gz
Parquet: fhv_tripdata_2019-03.parquet
GCS: fhv/fhv_tripdata_2019-03.parquet
Removing locally:
fhv_tripdata_2019-03.csv.gz
fhv_tripdata_2019-03.parquet
Local: fhv_tripdata_2019-04.csv.gz
Parquet: fhv_tripdata_2019-04.parquet
GCS: fhv/fhv_tripdata_2019-04.parquet
Removing locally:
fhv_tripdata_2019-04.csv.gz
fhv_tripdata_2019-04.parquet
Local: fhv_tripdata_2019-05.csv.gz
Parquet: fhv_tripdata_2019-05.parquet
GCS: fhv/fhv_tripdata_2019-05.parquet
Removing locally:
fhv_tripdata_2019-05.csv.gz
fhv_tripdata_2019-05.parquet
Local: fhv_tripdata_2019-06.csv.gz
Parquet: fhv_tripdata_2019-06.parqu